In [1]:
import pandas as pd
import json
from tqdm import tqdm
from pathlib import Path
cwd = Path('.')

In [2]:
'''Read in Raw Data: Comments.csv'''

comments_df= pd.read_csv(cwd/'output'/'comments.csv',error_bad_lines=False)
videos_df=pd.read_csv(cwd/'output'/'videos.csv', error_bad_lines=False)
comments_df=pd.merge(comments_df, videos_df, on='video_id',how='left')
comments_df.drop_duplicates(inplace=True)

comments_df.head(5)

C:\Users\AJL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,video_id,comment_text,likes_x,replies,title,channel_title,category_id,tags,views,likes_y,...,thumbnail_link,date,likes_log,views_log,dislikes_log,comment_log,category_name,like_rate,dislike_rate,comment_rate
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,...,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,12.676245,15.295757,8.688117,10.741730,Entertainment,7.283816,0.134979,1.052451
1,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,5457497,349857,...,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,14.09,12.765283,15.512501,9.768813,10.868264,Entertainment,6.410576,0.320275,0.961668
2,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,5854127,361422,...,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,15.09,12.797804,15.582658,9.999888,10.909107,Entertainment,6.173798,0.376196,0.933888
3,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,5997736,366401,...,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,16.09,12.811486,15.606893,10.038281,10.587266,Entertainment,6.108988,0.381561,0.660683
4,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,6109026,369963,...,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,17.09,12.821161,15.625278,10.052252,10.567489,Entertainment,6.056006,0.379881,0.635944


In [3]:
'''Map category name into the dataframe'''

with open(cwd/'data_scraper'/'kaggle'/'GB_category_id.json') as json_file:
    gb_json = json.load(json_file)

id_to_category = {}
for i in gb_json['items']:
    category = i['snippet']['title']
    id = int(i['id'])
    id_to_category[id] = category
training_id = []
for i in id_to_category:
    training_id.append(i)

In [ ]:
comments_df['category']=comments_df['category_id'].map(id_to_category)
comments_df = comments_df[comments_df['category_id']!= 29]
comments_df = comments_df[comments_df['category_id']!= 43]

main_df = []
for ID in tqdm(training_id):
    this_category = []
    this_category.append(ID)
    this_category.append(id_to_category[ID])
    comment_str = ''
    this_category_comment = comments_df[comments_df['category_id']==ID]
    for index,row in this_category_comment.iterrows():
        try:
            comment_str = comment_str + this_category_comment.at[index,'comment_text']
        except:
            continue
    this_category.append(comment_str)
    main_df.append(this_category)

df = pd.DataFrame(main_df,columns= ['id','category','comments'])

  0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
df

In [ ]:
'''Select only those categories with large enough data'''

df=df.loc[[0,1,2,3,4,6,7,9,10,11,12,13,14,15]]

# Cleaning the text

In [ ]:
import re
import string

df['comments'] = df['comments'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
'''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers'''
def clean_text(text):
    text= text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text = re.sub('\w*\d\w*','',text)
    return text
clean1= lambda x: clean_text(x)


df_clean = pd.DataFrame (df['comments'].apply(clean1))
df_clean['category'] = df['category']
df_clean=df_clean[['category','comments']]


In [ ]:
df_clean

In [ ]:
## Save to pickle file and open using pickle 

df_clean = pd.read_pickle ('df_clean.pkl')

In [ ]:
'''Produce a document term matrix of words for each category '''
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(stop_words = 'english')
data_cv = cv.fit_transform(df_clean.comments)
data_dtm = pd.DataFrame(data_cv.toarray(),columns=cv.get_feature_names())
data_dtm['category'] = df_clean['category']
data_dtm = data_dtm.set_index('category')
data_dtm

# Let's visualize the results

In [ ]:
# NUMBER 1 : WORD MAPPP
data=data_dtm.transpose()
data.head()

In [ ]:
'''Create dictionary to count top 30 words in each category'''
top_dict = {}
for c in data.columns:
    try:
        top = data[c].sort_values(ascending = False).head(30)
        top_dict[c] = list(zip(top.index, top.values))
    except:
        continue
top_dict

In [ ]:
words = []

for category in data.columns:
    try:
        top = [word for (word, count) in top_dict[category]]
    except:
        # some categories has less than 30 top words
        pass
    for t in top:
        words.append(t)
words   

In [ ]:
'''Show words which appear most commonly across categories'''

from collections import Counter
Counter(words).most_common()

In [ ]:
'''Make stop words include those words which appear most commonly in over half the categories'''

add_stop_words = [word for word, count in Counter(words).most_common() if count > 7]
add_stop_words

In [ ]:
'''Re-construct the dtm with new stop words'''

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

stop_words = text.ENGLISH_STOP_WORDS.union (add_stop_words)
cv=CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(df_clean.comments)
data_stop = pd.DataFrame (data_cv.toarray(),columns=cv.get_feature_names())

data_stop['category']=df['category']
data_stop = data_stop.set_index('category')
data_stop

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)


import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [16, 6]

cat_names = df_clean['category'].tolist()

df_clean.set_index('category',inplace=True)
df_clean

In [ ]:
for index, cat in enumerate(data.columns):
    try:
        wc.generate(df_clean.comments[cat])
    except:
        pass
    plt.subplot(4, 4, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(cat_names[index])
    
plt.show()

In [ ]:
# NUMBER 2: Amount of profanity
data_bad_words = pd.DataFrame (data= data_stop.transpose())
data_bad_words = data_bad_words.loc[['fucking','fuck', 'shit']]
data_bad_words = data_bad_words.transpose()
data_bad_words['f_word'] = data_bad_words['fucking']+data_bad_words['fuck']
data_bad_words

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]

for i in range(len(cat_names)):
    x = data_bad_words.f_word.loc[cat_names[i]]
    y = data_bad_words.shit.loc[cat_names[i]]
    plt.scatter(x, y, color='blue')
    plt.text(x+1.5, y+0.5, cat_names[i], fontsize=10)
    plt.xlim(-5, 5000) 
    
plt.title('Number of Bad Words', fontsize=20)
plt.xlabel('Number of F Bombs', fontsize=15)
plt.ylabel('Number of S Words', fontsize=15)

plt.show()